In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer, TargetEncoder
from sklearn.model_selection import train_test_split

In [3]:
df_train = pd.read_csv('train.csv')
X, y = df_train.drop(['SalePrice', 'Id'], axis=1), df_train.SalePrice
df_test = pd.read_csv('test.csv')

###Класс для подготовки данных:

In [4]:
class Data():
    """
    Класс для подготовки датасета к обучению/тестированию.
    Включает в себя генерацию признаков, заполнение пропусков.
    """

    def __init__(self, data: pd.DataFrame, is_train: bool=True):
        self.data = data.copy()
        self.is_train = is_train

        self.categorical = self.data.select_dtypes(exclude='number').columns
        self.numerical = self.data.select_dtypes(exclude='object').columns

        #ниже перечислены вещественные признаки,
        #которые будут переведены в бинарные
        self.zero_threshold       =   ['OpenPorchSF',
                                      'EnclosedPorch',
                                      '3SsnPorch',
                                      'ScreenPorch',
                                      'LowQualFinSF']

        self.one_threshold        =   'KitchenAbvGr'


        self.mlb = MultiLabelBinarizer()
        self.mlb.fit(df_train[['BsmtFinType1',
                               'BsmtFinType2',
                               'Exterior1st',
                               'Exterior2nd']]\
                .fillna('NO').to_numpy())


    def _QC_encoder(self, qual: str) -> int:
        """
        Метод для кодировки порядкого значения качества (str)
        в целое число.
        """

        qc = {
            'Po': 1,
            'Fa': 2,
            'TA': 3,
            'Gd': 4,
            'Ex': 5
        }

        if qual not in qc:
            return 0
        return qc[qual]


    def _encode_exposure(self, value: str) -> int:
        """
        Метод для кодировки порядкого значения (str) с 3 вариантами
        в целое число.
        """

        qc = {
              'Mn': 1,
              'Av': 2,
              'Gd': 3
          }

        if value not in qc:
            return 0
        return qc[value]


    def _encode_garage(self, value: str) -> int:
        """
        Метод для кодировки порядкого значения (str)
        степени завершенности гаража в целое число.
        """

        encoder = {
            'Unf': 1,
            'RFn': 2,
            'Fin': 3
        }

        if value not in encoder:
            return 0
        return encoder[value]


    def _multilabel(self):
        """
        Метод превращает признаки, которые могут содержать одновременно
        больше одной категории, в вектор из признаков, значения которых
        равны 0 или 1.
        """

        self.data[self.mlb.classes_] = \
        self.mlb.transform(
            self.data[['BsmtFinType1',
                       'BsmtFinType2',
                       'Exterior1st',
                       'Exterior2nd']].to_numpy())

        self.data.drop(['BsmtFinType1',
                        'BsmtFinType2',
                        'Exterior1st',
                        'Exterior2nd'], axis=1, inplace=True)


    def _encode_order(self):
        """
        Метод, объединяющий все кодирования порядковых переменных.
        """

        to_QC_encode = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
                    'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual',
                    'GarageCond']

        self.data[to_QC_encode]       = self.data[to_QC_encode]\
                                        .applymap(self._QC_encoder)
        self.data['BsmtExposure']     =  self.data['BsmtExposure']\
                                        .apply(self._encode_exposure)
        self.data['GarageFinish']     =  self.data['GarageFinish']\
                                        .apply(self._encode_garage)



    def _fill_missing(self):
        """
        Метод, заменяющий пропуски в категориальных переменных на 'NO'
        (отсутствие чего-либо) и в вещественных переменных на 0
        (тоже встречается при отсутствии того или иного атрибута жилья).
        """

        self.data[self.categorical] = self.data[self.categorical].fillna('NO')
        self.data[self.numerical] = self.data[self.numerical].fillna(0)


    def _drop_features(self):
        """
        Удаление переменных, которые были признаны неважными в ходе EDA и
        кросс-валидации.
        """

        to_drop = ['PoolArea',
                   'GarageCond',
                   'MoSold',
                   'YrSold',
                   'GarageArea',
                   'TotRmsAbvGrd',
                   'MiscVal',
                   'BsmtFinSF2',
                   'Street',
                   'Utilities',
                   'RoofMatl',
                   'Condition2',
                   'Id',
                   'LandSlope',
                   'BsmtCond',
                   ]

        self.data.drop(to_drop, inplace=True, axis=1)

    def _year_compare(self, row: pd.Series, value: str) -> int:
        """
        Метод для бинаризации года постройки гаража/ремонта.

        return: 1, если позже года основания дома, 0 в ином случае
        """

        return int(row[value] > row.YearBuilt)


    def _remod_bin(self, row: pd.Series) -> int:
        """
        Метод для бинаризации года ремонта.

        return: 1, если гараж построен не в тот же год, что и дом, 0 иначе
        """

        return self._year_compare(row, 'YearRemodAdd')


    def _garage_bin(self, row: pd.Series) -> int:
        """
        Метод для бинаризации года ремонта.

        return: 1, если ремонта не было (стоит тот же год, что и год постройки дома),
        0 иначе
        """

        return self._year_compare(row, 'GarageYrBlt')


    def _binarize(self):
        """
        Метод, объединяющий перевод некоторых переменных в двоичный вид.
        """

        self.data[self.zero_threshold] =  self.data[self.zero_threshold]\
                                          .applymap(lambda x: int(x > 0))

        self.data[self.one_threshold] =  self.data[self.one_threshold]\
                                          .apply(lambda x: int(x == 1))

        self.data['Heating']         =  self.data['Heating']\
                                          .apply(lambda x: int(x == 'GasA'))

        self.data['YearRemodAdd'] = self.data.apply(self._remod_bin, axis=1)
        self.data['GarageYrBlt'] = self.data.apply(self._garage_bin, axis=1)


    def _sum_FlrSf(self):
        """
        Генерация признака, представляющего собой сумму площадей обоих этажей.
        """

        self.data['SumFlrSf'] = self.data['2ndFlrSF'] + self.data['1stFlrSF']
        self.data.drop(['2ndFlrSF', '1stFlrSF'], axis=1, inplace=True)


    def preprocess(self, encode_categories: bool=False):
        """
        Проведение этапов препроцессинга.

        Если выборка обучающая, то из нее убираются выбросы.
        """

        self._encode_order()
        self._fill_missing()
        self._binarize()
        self._multilabel()
        self._sum_FlrSf()

        self._drop_features()
        numbers = self.data.select_dtypes(exclude='object').columns
        self.data[numbers] = self.data[numbers].astype(float)
        if self.is_train:
            self.data = self.data[self.data['LotArea'] < 50000]

In [105]:
df = Data(df_train)
df.preprocess()

train = df.data
X = train.loc[:, train.columns != 'SalePrice']
y = train.SalePrice

test = Data(df_test, is_train=False)
test.preprocess()

test = test.data

###Обучение модели

In [6]:
categorical_features_indices = np.where(X.dtypes != float)[0]

In [7]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)

In [8]:
!pip install catboost -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.9 MB/s eta 0:00:00


In [9]:
from catboost import CatBoostRegressor, Pool, metrics, cv
from sklearn.metrics import mean_squared_error as MSE

In [10]:
model = CatBoostRegressor(
    learning_rate=0.05,
    iterations=2000,
    max_depth=3,
    subsample=0.2,
    random_seed=42,
    l2_leaf_reg=2.0,
    logging_level='Silent'
)

#гиперпараметры подобраны при помощи кросс-валидации

In [11]:
cv_params = model.get_params()

cv_data = cv(
    Pool(X_train, y_train, cat_features=categorical_features_indices),
    cv_params
)


In [12]:
cv_data['test-RMSE-mean'][1995:]

1995    25876.411355
1996    25876.050987
1997    25878.089811
1998    25877.315788
1999    25877.474528
Name: test-RMSE-mean, dtype: float64

In [ ]:
model.fit(X_train, y_train,
          cat_features=categorical_features_indices,
          eval_set=(X_validation, y_validation),
          logging_level='Verbose')

In [14]:
def log_rmse(pred, test):
    return MSE(np.log(1 + pred), np.log(1 + test), squared=False)


log_rmse(model.predict(X_validation), y_validation)

0.12123951170972502

In [15]:
model.fit(X, y, cat_features=categorical_features_indices)

pd.DataFrame({'Id': df_test.Id, 'SalePrice': model.predict(test)}).to_csv('submission.csv', index=False)

In [72]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [20]:
cats = X.select_dtypes(exclude='number').columns

In [106]:
mean_encoder = TargetEncoder(cv=7, target_type='continuous')

mean_encoder.fit(X[cats], y)

X[cats] = mean_encoder.transform(X[cats])
test[cats] = mean_encoder.transform(test[cats])

<ipython-input-106-8a99c7992eea>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[cats] = mean_encoder.transform(X[cats])


In [181]:
xgb_regressor = XGBRegressor(n_estimators=2000,
                             subsample=0.15,
                             max_depth=4,
                             learning_rate=7.5e-3,
                             reg_lambda=10)

#гиперпараметры подобраны при помощи кросс-валидации

xgb_regressor.fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.0075, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=2000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [182]:
pd.DataFrame({'Id': df_test.Id, 'SalePrice': xgb_regressor.predict(test)}).to_csv('submission.csv', index=False)